In [1]:
import heapq
import osmnx as ox
import math
import pickle
import requests
from scipy.optimize import fmin_l_bfgs_b

In [2]:
f = open('blr.bin', 'rb')
print('Graph found, loading from disk')
G = pickle.load(f)
f.close()

Graph found, loading from disk


In [3]:
class a_star:
    def __init__(self, graph):
        self.graph = graph

    def get_route(self, pair1, pair2):
        self.start_node = ox.distance.nearest_nodes(self.graph, pair1[1], pair1[0])
        self.goal_node = ox.distance.nearest_nodes(self.graph, pair2[1], pair2[0])
        
        return self.__calculate_route()

    def __calculate_route(self):
        # Create a priority queue and add the start node with a cost of 0
        pq = [(0, self.start_node)]
        # Create a dictionary to keep track of the cost to reach each node
        cost_so_far = {self.start_node: 0}
        # Create a dictionary to keep track of the parent of each node
        parent = {self.start_node: None}
        
        while pq:
            # Pop the node with the lowest cost from the priority queue
            _, current_node = heapq.heappop(pq)
            
            # If we have reached the goal, reconstruct the path and return it
            if current_node == self.goal_node:
                path = []
                while current_node:
                    path.append(current_node)
                    current_node = parent[current_node]
                return path[::-1]
            
            # Otherwise, expand the current node's neighbors and add them to the priority queue
            for neighbor_node in self.__neighbors(current_node):
                # Calculate the cost to reach the neighbor node
                new_cost = cost_so_far[current_node] + 1
                # If we haven't visited this neighbor node yet or the new cost is lower than the old cost, update the cost and parent
                if neighbor_node not in cost_so_far or new_cost < cost_so_far[neighbor_node]:
                    cost_so_far[neighbor_node] = new_cost
                    priority = new_cost + self.__heuristic_cost_estimate(neighbor_node, self.goal_node)
                    heapq.heappush(pq, (priority, neighbor_node))
                    parent[neighbor_node] = current_node
        
        # If we get here, there is no path from the start node to the goal node
        return None
    def __haversine(self, pair1, pair2):
        """
        Calculate the great circle distance between two points
        on the earth (specified in decimal degrees)
        """
        # convert decimal degrees to radians
        lon1, lat1, lon2, lat2 = map(math.radians, [pair1[1], pair1[0], pair2[1], pair2[0]])

        # haversine formula
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
        c = 2 * math.asin(math.sqrt(a))
        r = 6371 # Radius of earth in kilometers. Use 3956 for miles
        return c * r
    
    def __heuristic_cost_estimate(self, n1, n2) -> float:
        if isinstance(n1, int):
            n1 = self.graph.nodes[n1]['x'], self.graph.nodes[n1]['y']
        if isinstance(n2, int):
            n2 = self.graph.nodes[n2]['x'], self.graph.nodes[n2]['y']
        x1, y1 = n1
        x2, y2 = n2
        return self.__haversine((y1, x1), (y2, x2))

    def __neighbors(self, node):
        return list(self.graph.neighbors(node))

In [4]:
source_coord = (12.92451735558134, 77.55801655636051)
dest_coord = (12.925299455248208, 77.56025666518755)

In [5]:
start_node = ox.distance.nearest_nodes(G, source_coord[1], source_coord[0])
goal_node = ox.distance.nearest_nodes(G, dest_coord[1], dest_coord[0])

In [6]:
routingObj = a_star(G)
path = routingObj.get_route(source_coord, dest_coord)

In [3]:
some_node = ox.distance.nearest_nodes(G, 77.640749, 12.889517)
print(some_node)

6757027803


In [6]:
print(G.nodes[some_node]['y'], G.nodes[some_node]['x'])

12.8893212 77.6402633


In [14]:
u, v = tuple(G.edges.items())[966]
print('Source:', u[0])
print('Dest:', u[1])
print('Distance:', v['length'])

Source: 248810128
Dest: 1466350486
Distance: 44.955


In [16]:
G.get_edge_data(248810128, 1466350486)[0]['length']

44.955

In [50]:
# Define the cost function
def cost_function(path, G):
    # Calculate the cost of the path
    # (e.g., based on path length, terrain, obstacles, etc.)
    cost = 0
    for i in range(len(path)-1):
        source = round(path[i])
        target = round(path[i+1])
        print(source, target)
        source = node_id_to_num[source]
        target = node_id_to_num[target]
        edge_data = G.get_edge_data(source, target)
        if edge_data:
            edge_cost = edge_data[0]['length']
        else:
            edge_cost = 191919191.0
        cost += edge_cost
    return cost

In [36]:
node_id_to_num = {i: node_id for i, node_id in enumerate(G.nodes)}

In [39]:
len(node_id_to_num)
for i in node_id_to_num.keys():
    if type(i) == type(1.0):
        print('aha')

In [28]:
G.get_edge_data(10784153212,10784153212)

In [24]:
[(0, G.number_of_nodes()-1)]*len(path)

[(0, 154309),
 (0, 154309),
 (0, 154309),
 (0, 154309),
 (0, 154309),
 (0, 154309),
 (0, 154309),
 (0, 154309)]

In [51]:
result = fmin_l_bfgs_b(lambda x: cost_function(x, G), path, bounds=[(0, G.number_of_nodes()-1)]*len(path), approx_grad=True)

4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4


In [52]:
print(result[0])

[4. 4. 4. 4. 4. 4. 4. 4.]


In [ ]:
fig, ax = ox.plot_graph_route(G, path, route_color='r', node_size = 0)

In [38]:
print(path)

None


In [28]:
routingObj = a_star(G)
routingObj.get_traffic_rate(12.931050, 77.542775)

0.8421052631578947

In [11]:
def get_traffic_rate(lat, lng):
    try:
        traffic = requests.get('https://api.tomtom.com/traffic/services/4/flowSegmentData/relative0/10/json??', params={"point" : str(lat) + ',' + str(lng), "unit" : "KMPH", 'openLr' : 'false', 'jsonp' : 'json', 'key' : 'G3zOAIIsnAZvOVniGeT38kZPJuTfFNaX'}).json()
        res = float(traffic['flowSegmentData']['currentSpeed']) / float(traffic['flowSegmentData']['freeFlowSpeed'])
        return res
    except:
        return 1

In [13]:
trafficData = dict()
print(len(G.nodes.items()))
for k, v in G.nodes.items():
    trafficData[k] = get_traffic_rate(v['y'], v['x'])

154310


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [15]:
154310 / 2500

61.724